### To Do
- Add a hash to every ActionObject
- Make every ActionObject that's created from other ActionObjects have an automatic hash 

- Bonus: 
    - Maybe see if we can stitch together Code Verification and Code Submission- enable the "Trace" feature!
    - Check if ActionObjects work with the dataset API

### Scenario

a = ActionObject(data1)\
a.history_hash = None\
b = ActionObject(data2)\
b.history_hash = None\
c = a + b\
c.history_hash = hash(a, b, add)\
d = a + b\
d.history_hash = hash(a, b, add)\
c.history_hash == d.history_hash\

e = a * b\
e.history_hash = hash(a, b, mul)

f = ActionObject(data1)
g = f + b



### Takeaways:
- Different ops with the same ActionObjects should have different hashes
(e and c should not have the same hash history)
- The same ActionObjects and op should always produce the same hash
(c and d should have the same hash history)
- Different ActionObjects with the same value should still have different hashes. (Check with Madhava)
(c and g should have different hash history because f might refer to the same data but in a different dataset/asset)

### Troubles

- Will we run into trouble with np.add vs torch.add vs regular add? \shrug

Do we really need hashes for this to work? Could we use f"{parent ID}_{OPname}"
- Could get gigantic strings if we're adding a billion things?



In [1]:
from syft.core.node.new.action_object import ActionObject

Potato gonna potate
Potato gonna potate
Potato gonna potate
Potato gonna potate


In [2]:
import numpy as np

In [3]:
data = 2*np.random.rand(10,10)-1

In [4]:
a = ActionObject(syft_action_data=data)
a.id

Lights... Cameras... ACTION OBJECT!
Potato gonna potate


<UID: b5ed23cc347b4bfba80207ea8ae018a9>

In [5]:
v = a.transpose()

Hey we got a name!  transpose
[<UID: b5ed23cc347b4bfba80207ea8ae018a9>]
We got a parent and their op!
-8504816534541149495  <--- this is my history hash
Lights... Cameras... ACTION OBJECT!
Potato gonna potate


In [6]:
v

History: -8504816534541149495

In [7]:
v.syft_history_hash

-8504816534541149495

In [8]:
v.syft_parent_id

<UID: 31c351e870774e73823034ff3581ebb1>

In [9]:
v.syft_parent_op

'transpose'

In [14]:
b = ActionObject(syft_action_data=data)
b.id

Lights... Cameras... ACTION OBJECT!
Potato gonna potate


<UID: 95d9fb6da2994db8bb6b3a926fdce14a>

In [9]:
b = ActionObject(syft_action_data=data ** 2)

Lights... Cameras... ACTION OBJECT!
Potato gonna potate


In [10]:
c = a + b
# c.id = hash(a.id + b.id + op.__name__)

Lights... Cameras... ACTION OBJECT!
Potato gonna potate
Lights... Cameras... ACTION OBJECT!
Potato gonna potate


In [5]:
hash(data)

TypeError: unhashable type: 'numpy.ndarray'

In [7]:
hash(data.tobytes())

2911586548322047132

In [ ]:
hash

In [23]:
hash(a.id)

929978662194457352

In [24]:
type(hash(a.id))

int

In [9]:
a == a

Lights... Cameras... ACTION OBJECT!
Lights... Cameras... ACTION OBJECT!
custom repr


array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])

In [19]:
c = a + 5

Lights... Cameras... ACTION OBJECT!


In [20]:
c.syft_parent_id

In [15]:
a.std()

Lights... Cameras... ACTION OBJECT!
custom repr


0.5851293838325092

In [10]:
type(a==a)

Lights... Cameras... ACTION OBJECT!
Lights... Cameras... ACTION OBJECT!


syft.core.node.new.action_object.ActionObject

In [8]:
b = a + a

Are we here?  _syft_output_action_object
Are we here?  __add__
Are we here?  __dict__
Are we here?  __private_attributes__
Are we here?  syft_action_data
Are we here?  syft_action_data
Are we here?  _syft_run_pre_hooks__
Are we here?  syft_pre_hooks__
Are we here?  __array_priority__
Are we here?  __dict__
Are we here?  __private_attributes__
Are we here?  syft_action_data
Are we here?  _syft_run_pre_hooks__
Are we here?  syft_pre_hooks__
Are we here?  _syft_run_post_hooks__
Are we here?  syft_post_hooks__
Are we here?  _syft_output_action_object
Are we here?  __class__
Are we here?  _init_private_attributes
Are we here?  __private_attributes__
Are we here?  _syft_set_validate_private_attrs_
Are we here?  __private_attributes__
Are we here?  __post_init__
Lights... Cameras... ACTION OBJECT!
Are we here?  __array_struct__
Are we here?  __dict__
Are we here?  __private_attributes__
Are we here?  syft_action_data
Are we here?  _syft_run_pre_hooks__
Are we here?  syft_pre_hooks__
Are we he

In [9]:
b.syft_parent_id

Are we here?  syft_parent_id


In [1]:
import syft as sy

In [2]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into angry_knuth... done!


In [3]:
domain.requests

""


In [4]:
type(domain.requests)

syft.core.node.domain_client.RequestQueueClient

In [11]:
from datetime import datetime

In [12]:
datetime.now()

datetime.datetime(2023, 1, 23, 15, 49, 48, 622675)

In [ ]:
from typing import Callable
import numpy as np

# This is our decorator
def submit_to_domain_node(user_func: Callable, user_input: np.ndarray, policy, constraint, output_args):
    print("We would try to check that the verify key is correct")
    print("We 

In [1]:
def parent(num):
    
    def first_child():
        return "Emily"
    
    def second_child():
        return "George"
    
    if num == 1:
        return first_child
    else:
        return second_child

In [5]:
def decorator(func):
    def wrapper():
        print("Something happens before the function")
        func()
        print("Something happens after the function")
    return wrapper

In [6]:
def say_whee():
    print("Whee!")

say_whee = decorator(say_whee)

In [7]:
say_whee()

Something happens before the function
Whee!
Something happens after the function


In [8]:
@decorator
def say_boo():
    print("Booo!")

In [10]:
say_boo()

Something happens before the function
Booo!
Something happens after the function
